In [8]:
import math
import numpy as np
import pandas as pd

In [5]:
!pip install pandas

  Using cached pandas-1.5.1-cp38-cp38-win_amd64.whl (11.0 MB)
  Using cached pytz-2022.5-py2.py3-none-any.whl (500 kB)


You should consider upgrading via the 'c:\git\machine-learning-from-scratch\venv\scripts\python.exe -m pip install --upgrade pip' command.


In [26]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

In [23]:
class NeuralNetwork():
    def __init__(self, x_shape, y_shape, n_node=4):
        self.weigths1 = np.random.rand(x_shape[1], n_node)
        self.weigths2 = np.random.rand(n_node, y_shape[1])
        self.output = np.zeros(y_shape[1])
    
    def feedforward(self, x):
        self.input = x
        self.layer1 = sigmoid(np.dot(self.input, self.weigths1))
        self.output = sigmoid(np.dot(self.layer1, self.weigths2))
        
    def backpropagation(self, y):
        self.y = y
        # application of the chain rule to find derivative of the loss function with respect to weights2 and weights1
        d_weights2 = np.dot(self.layer1.T, (2*(self.y - self.output) * sigmoid_derivative(self.output)))
        d_weights1 = np.dot(self.input.T,  (np.dot(2*(self.y - self.output) * sigmoid_derivative(self.output), self.weights2.T) * sigmoid_derivative(self.layer1)))

        # update the weights with the derivative (slope) of the loss function
        self.weights1 += d_weights1
        self.weights2 += d_weights2
    
    def calculate_loss(self):
        # using sum of square error
        self.loss = sum(np.dot((self.y - self.output), (self.y - self.output)))
    
    def train(self, x_data, y_data, epochs):
        for i in range(epochs):
            for i, x in enumerate(x_data):
                y = y_data[i]
                self.feedforward(x)
                self.backpropagation(y)
                self.calculate_loss
                print('epoch {} ==> loss: {.3f}'.format(i+1, self.loss))
    
    def predict(self, x):
        self.feedforward(x)
        return self.output

In [18]:
# For demonstration, we will use dummy data
# suppose that we want to model binary equation y = x1 and (x2 or x3)
x1 = [1, 0, 0, 1, 1, 1]
x2 = [1, 0, 1, 0, 0, 1]
x3 = [1, 0, 1, 1, 0, 0]
y = [1, 0, 0, 1, 0, 1]
dataset = pd.DataFrame({'x1': x1, 'x2': x2, 'x3': x3, 'y': y})
dataset

,x1,x2,x3,y
0,1,1,1,1
1,0,0,0,0
2,0,1,1,0
3,1,0,1,1
4,1,0,0,0
5,1,1,0,1


In [25]:
x_data = dataset[['x1', 'x2', 'x3']].to_numpy()
y_data = dataset['y'].to_numpy().reshape(-1, 1)
print('x_data:\n', x_data.shape)
print('y_data:\n', y_data.shape)

x_data:
 (3,)
y_data:
 (6, 1)


In [27]:
NN_model = NeuralNetwork(x_data.shape, y_data.shape, n_node=5)
NN_model.train(x_data, y_data, epochs=10)

ValueError: shapes (5,) and (1,) not aligned: 5 (dim 0) != 1 (dim 0)